##Bert Bi-GRU for sentiment analysis

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.0MB 8.4MB/s 
     |████████████████████████████████| 1.1MB 30.0MB/s 
     |████████████████████████████████| 890kB 58.8MB/s 
     |████████████████████████████████| 3.0MB 57.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=6137574d80daff10dc2ab173dcfce5c626c5c7024cd82d28bc9b127ce7805433
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch #pytorch library

import random
import numpy as np

SEED = 1234

random.seed(SEED) #setting random seed
np.random.seed(SEED) #setting 
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
from transformers import BertTokenizer
#loading the pre-trained bert-base-uncased tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
#checking number of tokens in BERT vocabulary
len(tokenizer.vocab)

30522

In [ ]:
#tokenizing using bert tokenizer
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

['hello', 'world', 'how', 'are', 'you', '?']


In [ ]:
#numericalizing tokens
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[7592, 2088, 2129, 2024, 2017, 1029]


In [ ]:
init_token = tokenizer.cls_token # first token of the sequence 
eos_token = tokenizer.sep_token # last token of a sequence 
pad_token = tokenizer.pad_token # token used for padding
unk_token = tokenizer.unk_token # unknown token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [ ]:
# the indexes of the special tokens by converting them using the vocabulary
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

# or we can get these indexes by explicitly getting them from the tokenizer.
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100
101 102 0 100


In [ ]:
# the model was trained on sequences with a defined maximum length. 
# Getting this maximum length.
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


In [ ]:
# making our own tokenizing function that tokenizes sentences and cuts them to a 
# size (max_input_length - 2) 
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving amazon_cells_labelled.txt to amazon_cells_labelled.txt


In [ ]:
import pandas as pd
dataset = pd.read_csv('/content/amazon_cells_labelled.txt', sep="\t", names=["Review", "Score"])

In [ ]:
dataset

,Review,Score
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
...,...,...
995,The screen does get smudged easily because it ...,0
996,What a piece of junk.. I lose more calls on th...,0
997,Item Does Not Match Picture.,0
998,The only thing that disappoint me is the infra...,0


In [ ]:
dataset.to_csv('dataset.csv')

In [ ]:
from torchtext import data
#creating preprocessing pipelines for text and label columns
Review = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx,lower=True)

Score = data.LabelField(dtype = torch.float)

In [ ]:
fields = {'Review': ('r', Review), 'Score': ('s', Score)}

In [ ]:
#importing dataset
from torchtext import datasets
train_data = data.TabularDataset(path = '/content/dataset.csv',format = 'csv',fields=fields)
#splitting train_data to train data and valid data
train_data, test_data = train_data.split(random_state = random.seed(SEED),split_ratio=0.8)

In [ ]:
train_data, valid_data = train_data.split(random_state = random.seed(SEED),split_ratio=0.8)

In [ ]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 640
Number of validation examples: 160
Number of testing examples: 200


In [ ]:
print(vars(train_data.examples[6]))

{'r': [1996, 2640, 2003, 2200, 5976, 1010, 2004, 1996, 4540, 1000, 12528, 1000, 2003, 2025, 2200, 6625, 2012, 2035, 1012], 's': '0'}


In [ ]:
# getting sentence from list of indexes
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['r'])

print(tokens)

['the', 'design', 'is', 'very', 'odd', ',', 'as', 'the', 'ear', '"', 'clip', '"', 'is', 'not', 'very', 'comfortable', 'at', 'all', '.']


In [ ]:
#building label vocabulary
Score.build_vocab(train_data)

In [ ]:
print(Score.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f4ca7e3f2f0>, {'1': 0, '0': 1})


In [ ]:
#creating iterator for train, valid and test data
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.r), 
    sort_within_batch=True,
    device = device)

In [ ]:
from transformers import BertTokenizer, BertModel
#importing the pretrained bert model
bert = BertModel.from_pretrained('bert-base-uncased')

Building our model

In [ ]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        #we use bert pretrained embeddings
        embedding_dim = bert.config.to_dict()['hidden_size']
        #defining our GRU layer 
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        #defining fully connected layer
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        #defining dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            # The transformer returns the embeddings for the whole sequence 
            # as well as a pooled output. We require the embeddings
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [ ]:
#defining hyperparameters
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25
# creating the model instance
model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [ ]:
#getting the number of parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters


In [ ]:
# we set the requires_grad to false for all the bert parameters because they are pre trained
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [ ]:
#this reduces our number of trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,759,169 trainable parameters


In [ ]:
# checking the names of the trainable parameters, ensuring they make sense. 
# As we can see, they are all the parameters of the GRU (rnn) and the linear layer (out).
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


In [ ]:
#importing optim library to define our optimizer
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
#defining our loss function
criterion = nn.BCEWithLogitsLoss()

In [ ]:
# putting model and criterion to device
model = model.to(device)
criterion = criterion.to(device)

Training our model

In [ ]:
#defining our accuracy function
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    """ training our model"""    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.r).squeeze(1)
        
        loss = criterion(predictions, batch.s)
        
        acc = binary_accuracy(predictions, batch.s)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    """ evaluating our model on validation dataset"""    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.r).squeeze(1)
            
            loss = criterion(predictions, batch.s)
            
            acc = binary_accuracy(predictions, batch.s)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    """calculating how long a training/evaluation epoch takes"""
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
#training our model
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    # saving the models which have better valid_loss    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 0.667 | Train Acc: 58.13%
	 Val. Loss: 0.927 |  Val. Acc: 48.44%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.424 | Train Acc: 85.00%
	 Val. Loss: 0.360 |  Val. Acc: 83.33%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.299 | Train Acc: 88.75%
	 Val. Loss: 0.366 |  Val. Acc: 81.25%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.248 | Train Acc: 89.38%
	 Val. Loss: 0.322 |  Val. Acc: 86.46%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.200 | Train Acc: 92.50%
	 Val. Loss: 0.347 |  Val. Acc: 83.33%
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.166 | Train Acc: 93.59%
	 Val. Loss: 0.347 |  Val. Acc: 84.38%
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.134 | Train Acc: 94.84%
	 Val. Loss: 0.623 |  Val. Acc: 80.21%
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.196 | Train Acc: 92.66%
	 Val. Loss: 0.618 |  Val. Acc: 78.65%
Epoch: 09 | Epoch Time: 0m 0s
	Train Loss: 0.215 | Train Acc: 91.88%
	 Val. Loss: 0.408 |  Val. Acc: 82.81%
Epoch: 10 | Epoch Time: 0m 0

In [ ]:
#loading the model with minimum valid_loss
model.load_state_dict(torch.load('tut6-model.pt'))
# testing on our test_data
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.267 | Test Acc: 91.02%


In [ ]:
def predict_sentiment(model, tokenizer, sentence):
    """ returns the prediction of model on the input sentence using the model"""
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    if prediction.item()>=0.5:
        print('Negative with score '+str(1-prediction.item()))
    else:
        print('Positive with score '+str(1- prediction.item()) )     

In [ ]:
predict_sentiment(model, tokenizer, "Highly recommend for any one who has a blue tooth phone.")

Positive with score 0.9905686117708683


In [ ]:
predict_sentiment(model, tokenizer, "I advise EVERYONE DO NOT BE FOOLED!")

Negative with score 0.22611218690872192


In [ ]:
predict_sentiment(model, tokenizer, "This product is cheap,but it is not upto my expectation")

Negative with score 0.4909631609916687


In [ ]:
predict_sentiment(model, tokenizer, "This product is not upto my expectation")

Negative with score 0.10951513051986694


In [ ]:
predict_sentiment(model, tokenizer, "Amazing product but price is high")

Positive with score 0.7733954638242722


In [ ]:
predict_sentiment(model, tokenizer, "Does really an amazing product as price is high")

Positive with score 0.9859268562868237
